In [1]:
import numpy as np
import pylhef
import sys
from decimal import Decimal


In [2]:
lhfile = pylhef.read('test_events.lhe')

sim_events = []
eventlength = []
tqlist=[]

for event in lhfile.events:       # iterates through all events, modifies existing charactersistics of top quark
    
    tq = 0
    particles=event.particles      #finds the index of the top quark
    eventlength.append(len(particles))
    for particle in particles:
        if particle.id == 6:
            tqlist.append(tq)
            break
        else:
            tq += 1
    
   # defines values for each entry in simulated event
    part = event.particles[tq] 
    pid = part.id
    inout = 1
    fmom = tq+1
    lmom = tq+1
    col1 = part.color[0]
    col2 = part.color[1]
    px = part.p[1]/2
    py = part.p[2]/2
    pz = part.p[3]/2
    energy = part.p[0]/2
    mas = part.mass
    lt = part.lifetime
    spi = part.spin
                      
    # converts all inputs to strings, converts relevant items to scientific notation
    s_part = str(part)
    s_pid = str(pid)
    s_inout = str(inout)
    s_fmom = str(fmom)
    s_lmom = str(lmom)
    s_col1 = str(col1)
    s_col2 = str(col2)
    s_px = '%.10e' % Decimal(px)
    if px > 0:
        s_px='+' + s_px
    s_py = '%.10e' % Decimal(py)
    if py > 0:
        s_py = '+' + s_py
    s_pz = '%.10e' % Decimal(pz)
    if pz > 0:
        s_pz = '+' + s_pz
    s_energy = '%.10e' % Decimal(energy)
    s_mas = '%.10e' % Decimal(mas) 
    s_lt = '%.4e' % Decimal(lt)
    s_spi = '%.4e' % Decimal(spi)    
    
    simpart = [s_pid,s_inout,s_fmom,s_lmom,s_col1,s_col2,s_px,s_py,s_pz,s_energy,s_mas,s_lt,s_spi]        
    sim_events.append(simpart)                      # writes simulated particle to entry in list


In [3]:
# changes num values of simulated particles to string

sim_events[0]     
sim_events_str = []
for n in range(len(sim_events)):
    #string = ' '.join(str(e) for e in sim_events[n]),'\n'
    ev = sim_events[n]
    string = "%9d %2d %4s %4s %4s %4s %3s %3s %3s %3s  %2s %3s %3s\n" % (int(ev[0]),int(ev[1]),ev[2],ev[3],ev[4],ev[5],ev[6],ev[7],ev[8],ev[9],ev[10],ev[11],ev[12])
    
    sim_events_str.append(string)

#print(sim_events_str)

In [4]:
test=open('test_events.lhe','r')
file=[]

output = ""
for line in test:
    file.append(line)

# finds line number in original lhe file where each event starts
lines=[i for i,x in enumerate(file) if x=='<event>\n']
len(lines) # = 94

toplist=[]
for u in range(len(lines)):
    top = lines[u]+2+tqlist[u]
    topparticle = file[top]
    topparticle = topparticle.replace(" 6  1 "," 6  2 ")
    toplist.append(topparticle)
    
    
sims=[]
n=0
while n<=len(lines):
    if n==0:                               # writes header up to first event as first entry in list sims
        place = lines[n]+2+eventlength[n]    ## writes first event and first simulated particle to list
        topplace = lines[n]+2+tqlist[n]
        sims.append(file[0:topplace])
        sims.append(toplist[n])
        sims.append(file[topplace+1:place])
        sims.append(sim_events_str[n])
    elif n < len(lines):                  # writes each event and respective simulated particle to list
        place = lines[n]+2+eventlength[n]
        topplace = lines[n]+2+tqlist[n]
        last = lines[n-1]+2+eventlength[n-1]
        sims.append(file[last:topplace])
        sims.append(toplist[n])
        sims.append(file[topplace+1:place])
        sims.append(sim_events_str[n])
    else:                                      # writes last few lines of last event to list
        last = lines[n-1]+2+eventlength[n-1]
        sims.append(file[last:len(file)-1])
    n=n+1
    
head = sims[0][0:-6]               # establishes header
foot = '</LesHouchesEvents>'       # establishes footer


In [5]:
# prepares header and footer in proper form
header = ('').join(head)
e1 = ('').join(sims[0][-6:last])

# writes strings to new file
f = open('simulated_event.lhe', 'w')
f.write(header)
f.write(e1)

for n in range(1,len(sims)):       # iterates entries of sims (excluding header/footer), writes the strings to file
    fix = ('').join(sims[n])
    f.write(fix)

f.write(foot)

f.close()


Myrl, can you figure out what all this stuff is and make notes of it here. 




In [6]:
'''
<event>
 4      1 +5.0558200e+02 2.08121300e+02 7.54677100e-03 1.14306200e-01
       21 -1    0    0  503  502 +0.0000000000e+00 +0.0000000000e+00 +4.6699672078e+02 4.6699672078e+02 0.0000000000e+00 0.0000e+00 -1.0000e+00
       21 -1    0    0  501  503 -0.0000000000e+00 -0.0000000000e+00 -1.7306846547e+02 1.7306846547e+02 0.0000000000e+00 0.0000e+00 -1.0000e+00
        6  1    1    2  501    0 -1.0549343684e+02 +4.7503664050e+01 +3.6498091942e+02 4.2014941967e+02 1.7300000000e+02 0.0000e+00 -1.0000e+00
       -6  1    1    2    0  502 +1.0549343684e+02 -4.7503664050e+01 -7.1052664100e+01 2.1991576658e+02 1.7300000000e+02 0.0000e+00 -1.0000e+00
<mgrwt>
<rscale>  2 0.20812127E+03</rscale>
<asrwt>0</asrwt>
<pdfrwt beam="1">  1       21 0.71845649E-01 0.20812127E+03</pdfrwt>
<pdfrwt beam="2">  1       21 0.26625918E-01 0.20812127E+03</pdfrwt>
<totfact> 0.23273598E+04</totfact>
</mgrwt>
</event>
''';

In [7]:
### To do: 
#
#  lines containing simulated particle events need to be formatted:
##      fix spacing
##      add signs on positive decimal values  


# For the columns
# PID
# Input (-1), temporary (2), final state (1)
# Next is first mother. For a particle *created* in a collision, there will be two values. This is the index and the index starts at 1
# Next is second mother. For a particle *created* in a collision, there will be two values. This is the index and the index starts at 1
# Color charge
# Anticolor charge
# px
# py
# pz
# Energy
# Mass
# Displaced vertex
# Helicity






In [8]:
## Sample showing some decays
#
'''
#<event>
#11      1 +2.0701000e-05 2.31280300e+02 7.81860800e-03 1.13300500e-01
#       21 -1    0    0  505  504 +0.0000000000e+00 +0.0000000000e+00 +1.4094688243e+02 1.4094688243e+02 0.0000000000e+00 0.0000e+00 1.0000e+00
#       21 -1    0    0  503  505 -0.0000000000e+00 -0.0000000000e+00 -4.0829194094e+02 4.0829194094e+02 0.0000000000e+00 0.0000e+00 -1.0000e+00
#        6  2    1    2  503    0 +1.4618203726e+02 +4.7895145873e+01 -2.0961478162e+02 3.1213608086e+02 1.7270623406e+02 0.0000e+00 0.0000e+00
#       -6  2    1    2    0  504 -1.4618203726e+02 -4.7895145873e+01 -5.7730276890e+01 2.3710274251e+02 1.7094382884e+02 0.0000e+00 0.0000e+00
#      -24  2    4    4    0    0 -4.6911039134e+01 +3.4432144994e+01 -2.8185581484e+01 1.0471130428e+02 8.2363900094e+01 0.0000e+00 0.0000e+00
#       -5  1    3    3    0  501 +7.6319226347e+01 -1.0790619659e+01 -1.4288904741e+02 1.6242053951e+02 4.7000000000e+00 0.0000e+00 -1.0000e+00
#       -4  1    3    3    0  502 +2.9928146279e+01 -1.5327064351e+01 -5.5449699099e+01 6.4863690702e+01 1.4200000000e+00 0.0000e+00 1.0000e+00
#      -13  1    3    3    0    0 +3.9934664628e+01 +7.4012829883e+01 -1.1276035107e+01 8.4851850646e+01 1.0566000000e-01 0.0000e+00 1.0000e+00
#      -12  1    5    5    0    0 -3.8802536483e+01 -6.2616821453e+00 +1.7474356577e+01 4.3013935400e+01 0.0000000000e+00 0.0000e+00 1.0000e+00
#       11  1    5    5    0    0 -8.1085026506e+00 +4.0693827139e+01 -4.5659938062e+01 6.1697368878e+01 5.1100000000e-04 0.0000e+00 -1.0000e+00
#       -5  1    4    4    0  504 -9.9270998121e+01 -8.2327290867e+01 -2.9544695405e+01 1.3239143823e+02 4.7000000000e+00 0.0000e+00 1.0000e+00
#<mgrwt>
''';